<a href="https://colab.research.google.com/github/sakshi170920/Public-Transport-System-Optimization-/blob/main/Distance_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Loading & Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/

gtfs_data = "googletransitfeed.zip";

Mounted at /content/drive
/content/drive/My Drive


In [1]:
!pip install geopandas
from zipfile import ZipFile
import pandas as pd
import geopandas as gpd
import datetime
import numpy as np
import matplotlib.colors as colors

     |████████████████████████████████| 1.0 MB 29.5 MB/s 
     |████████████████████████████████| 6.3 MB 46.1 MB/s 
     |████████████████████████████████| 16.7 MB 346 kB/s 


In [ ]:
with ZipFile(gtfs_data) as myzip:
    shapes_df = pd.read_csv(myzip.open("shapes.txt"), dtype={
        'shape_id': 'str', 
        'shape_pt_lat': 'float', 
        'shape_pt_lon': 'float',  
        'shape_pt_sequence': 'Int64', 
        'shape_dist_traveled': 'float',
    })

    routes_df = pd.read_csv(myzip.open("routes.txt"), dtype={
        'route_id': 'str',  
        'agency_id': 'str',  
        'route_short_name': 'str',  
        'route_long_name': 'str', 
        'route_desc': 'str', 
        'route_type': 'Int64',
        'route_color': 'str',  
        'route_text_color': 'str', 
        'exact_times': 'bool'
    })
    
    trips_df = pd.read_csv(myzip.open("trips.txt"), dtype={
        'route_id': 'str', 
        'service_id': 'str',  
        'trip_id': 'str',
        'shape_id': 'str', 
        'trip_headsign': 'str', 
        'direction_id': 'str',  
        'block_id': 'str', 
        'wheelchair_accessible': 'str', 
        'route_direction': 'str', 
        'trip_note': 'str', 
        'bikes_allowed': 'str'
    })
    
    calendar_df = pd.read_csv(myzip.open("calendar.txt"), dtype={
        'service_id': 'str',  
        'monday': 'bool',  
        'tuesday': 'bool',  
        'wednesday': 'bool',  
        'thursday': 'bool',  
        'friday': 'bool', 
        'saturday': 'bool',  
        'sunday': 'bool',  
        'start_date': 'str', 
        'end_date': 'str',
    })
    
    calendar_dates_df = pd.read_csv(myzip.open("calendar_dates.txt"), dtype={
        'service_id': 'str',  
        'date': 'str',
        'exception_type': 'Int64',
    })

    stops_df = pd.read_csv(myzip.open("stops.txt"), dtype={
        'stop_id': 'str',  
        'stop_code': 'str',
        'stop_name': 'str',
        'stop_desc': 'str',
        'stop_lat': 'float',
        'stop_lon': 'float',
        'stop_url': 'str',
        'location_type': 'str',
        'parent_station': 'str',
        'stop_timezone': 'str',
        'wheelchair_boarding': 'str'
    })

In [ ]:
shapes_df

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence
0,1001,39.20101,-86.54726,0
1,1001,39.20154,-86.54730,1
2,1001,39.20226,-86.54732,2
3,1001,39.20244,-86.54732,3
4,1001,39.20280,-86.54733,4
...,...,...,...,...
30205,3042,39.16425,-86.53239,777
30206,3042,39.16426,-86.53365,778
30207,3042,39.16456,-86.53364,779
30208,3042,39.16454,-86.53237,780


####As shape_pt_lat and shape_pt_lon contain geographical coordinates we converted it into **geopandas GeoDataFrame with geometry column**. 

*ESPG 4326 indicates that coordinates contain geographical latitude and longitude in WGS-84 format.*

In [ ]:
shapes_gdf = gpd.GeoDataFrame(shapes_df, 
        geometry=gpd.points_from_xy(shapes_df.shape_pt_lon, shapes_df.shape_pt_lat)).set_crs(epsg=4326)
shapes_gdf

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,geometry
0,1001,39.20101,-86.54726,0,POINT (-86.54726 39.20101)
1,1001,39.20154,-86.54730,1,POINT (-86.54730 39.20154)
2,1001,39.20226,-86.54732,2,POINT (-86.54732 39.20226)
3,1001,39.20244,-86.54732,3,POINT (-86.54732 39.20244)
4,1001,39.20280,-86.54733,4,POINT (-86.54733 39.20280)
...,...,...,...,...,...
30205,3042,39.16425,-86.53239,777,POINT (-86.53239 39.16425)
30206,3042,39.16426,-86.53365,778,POINT (-86.53365 39.16426)
30207,3042,39.16456,-86.53364,779,POINT (-86.53364 39.16456)
30208,3042,39.16454,-86.53237,780,POINT (-86.53237 39.16454)


In [ ]:
stops_gdf = gpd.GeoDataFrame(stops_df, 
        geometry=gpd.points_from_xy(stops_df.stop_lon, stops_df.stop_lat)).set_crs(epsg=4326)
stops_gdf

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,geometry
0,2759346,45690,Departing Transit Center,"Transit Center bordered by E 3rd, S Walnut and...",39.164286,-86.533571,NaN,0,NaN,NaN,NaN,POINT (-86.53357 39.16429)
1,2927977,45650,Lincoln St and Kirkwood Ave,southeast corner over intersection by stop sign,39.166518,-86.531045,NaN,0,NaN,NaN,2,POINT (-86.53104 39.16652)
2,2927978,45505,Kirkwood Ave and Grant St (Outbound-Eastbound),southwest corner of intersection,39.166486,-86.529943,NaN,0,NaN,NaN,NaN,POINT (-86.52994 39.16649)
3,2927979,45143,Kirkwood Ave and Dunn St,southwest corner of intersection near stop sign,39.166483,-86.528359,NaN,0,NaN,NaN,2,POINT (-86.52836 39.16648)
4,2927980,45597,7th St and Woodlawn Ave at the Indiana Memoria...,located at safety island and protected bicycle...,39.168439,-86.523929,NaN,0,NaN,NaN,1,POINT (-86.52393 39.16844)
...,...,...,...,...,...,...,...,...,...,...,...,...
543,2977550,45121,Kirkwood Ave & Maple St,NaN,39.166595,-86.542139,NaN,0,NaN,NaN,NaN,POINT (-86.54214 39.16659)
544,2977551,45122,Kirkwood Ave & Jackson St IB,NaN,39.166553,-86.539645,NaN,0,NaN,NaN,NaN,POINT (-86.53964 39.16655)
545,2977552,45695,Kirkwood Ave @ B-Line Trail IB,NaN,39.166545,-86.536399,NaN,0,NaN,NaN,NaN,POINT (-86.53640 39.16655)
546,2977537,45737,Curry Pk @ Baxter Healthcare,NaN,39.159132,-86.582569,NaN,0,NaN,NaN,NaN,POINT (-86.58257 39.15913)


In [ ]:
calendar_df

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,95635,True,True,True,True,True,False,False,20220102,20220507
1,95637,False,False,False,False,False,True,False,20220102,20220507
2,95638,False,False,False,False,False,False,True,20220102,20220507
3,95636,True,True,True,True,True,True,False,20220102,20220507


###Finding which services run on specific date

In [ ]:
show_date_str = "2022-01-03"

date = datetime.datetime.strptime(show_date_str, "%Y-%m-%d")
date_string = date.strftime("%Y%m%d")
day_of_week_name = date.strftime('%A').lower()

services_for_day_1 = calendar_df[(calendar_df[day_of_week_name]) & (date_string >= calendar_df.start_date) & (date_string <= calendar_df.end_date)].service_id.to_numpy()
print(f"scheduled for day based on calendar: {len(services_for_day_1)}")

# exception_type
# 1 - Service has been added for the specified date.
# 2 - Service has been removed for the specified date.
services_added_for_day = calendar_dates_df[(calendar_dates_df.date == date_string) & (calendar_dates_df.exception_type == 1)].service_id.to_numpy()
services_removed_for_day = calendar_dates_df[(calendar_dates_df.date == date_string) & (calendar_dates_df.exception_type == 2)].service_id.to_numpy()
print(f"services added using calendar_dates: {len(services_added_for_day)}")
print(f"services removed using calendar_dates: {len(services_removed_for_day)}")

services_for_day_2 = np.concatenate([services_for_day_1, services_added_for_day])
services_for_day = np.setdiff1d(services_for_day_2, services_removed_for_day)

print(f"final services for day: {len(services_for_day)}")

scheduled for day based on calendar: 2
services added using calendar_dates: 0
services removed using calendar_dates: 0
final services for day: 2


In [ ]:
coords = shapes_df[["shape_pt_lat", "shape_pt_lon", "shape_pt_sequence"]]
coords_roll_1 = np.roll(coords, 1, axis=0)

segments = pd.DataFrame(np.concatenate([coords_roll_1, coords], axis=1), columns=["start_lat", "start_lng", "start_seq", 
                                                                       "end_lat", "end_lng", "end_seq"])
segments_df = shapes_df[["shape_id"]].join(segments)
segments_df = segments_df[segments_df.end_seq != 1]
segments_df = segments_df.drop(columns=['end_seq']).rename(columns={ "start_seq": "seq" })
segments_df

,shape_id,start_lat,start_lng,seq,end_lat,end_lng
0,1001,39.16456,-86.53237,781,39.20101,-86.54726
2,1001,39.20154,-86.5473,1,39.20226,-86.54732
3,1001,39.20226,-86.54732,2,39.20244,-86.54732
4,1001,39.20244,-86.54732,3,39.2028,-86.54733
5,1001,39.2028,-86.54733,4,39.20333,-86.54734
...,...,...,...,...,...,...
30205,3042,39.16455,-86.53237,776,39.16425,-86.53239
30206,3042,39.16425,-86.53239,777,39.16426,-86.53365
30207,3042,39.16426,-86.53365,778,39.16456,-86.53364
30208,3042,39.16456,-86.53364,779,39.16454,-86.53237


In [ ]:
routes_df.head()

,route_id,route_short_name,route_long_name,route_desc,route_url,route_color,route_text_color,route_type
0,102936,2 West,2 West 11th St via Showers Complex,NaN,NaN,035ebf,fafafa,3
1,103046,9,9 IU Campus / College Mall / Covenanter & Clarizz,NaN,NaN,05c5f5,fdfcfc,3
2,102905,3 West,3 Highland Village / Curry Pike,NaN,NaN,239504,f9f6f6,3
3,102906,4 South,4 High Street / Sherwood Oaks,NaN,NaN,b505f5,f8f7f7,3
4,102908,5,5 Sare Road,NaN,NaN,f50a68,faf9f9,3


In [ ]:
day_trips = trips_df[trips_df.service_id.isin(services_for_day)]
sydney_bus_route_ids = routes_df[routes_df.route_desc == "2 West 11th St via Showers Complex"].route_id.unique()
shape_day_trips = day_trips[day_trips.route_id.isin(sydney_bus_route_ids)].groupby(by="shape_id").size().to_frame("day_trips")
shape_day_trips

,day_trips
shape_id,


In [ ]:
routes_df['route_long_name'].value_counts()

2 West 11th St via Showers Complex                   1
9 IU Campus / College Mall / Covenanter & Clarizz    1
3 Highland Village / Curry Pike                      1
4 High Street / Sherwood Oaks                        1
5 Sare Road                                          1
1 South Walnut / Clear Creek Shoppping Center        1
3 College Mall / East 3rd Street                     1
4 Bloomfield Road                                    1
8 Eastside Local                                     1
1 Fee Lane BHS North                                 1
6 Campus Shuttle                                     1
6 Limited                                            1
7 Henderson / Walnut Express                         1
2 South Rogers                                       1
10 East 10th / IU Health Hospital                    1
Name: route_long_name, dtype: int64

In [ ]:
shape_segments_day_trips = pd.merge(segments_df, shape_day_trips, left_on="shape_id", right_index=True)

# how many trip occurs in each segment per day
shape_segments_day_trips = pd.merge(segments_df, shape_day_trips, left_on="shape_id", right_index=True)

In [ ]:
segment_day_trips = shape_segments_day_trips.groupby(by=[
    "start_lat", "start_lng", "end_lat", "end_lng"]).sum("day_trips").reset_index()
segment_day_trips  

,start_lat,start_lng,end_lat,end_lng,day_trips


In [ ]:
from shapely.geometry import Point, LineString

def get_line_string(row):
    start = Point(row.start_lng, row.start_lat)
    end = Point(row.end_lng, row.end_lat)
    line = LineString([start, end])
    row["geometry"] = line
    return row

segment_day_trips_gpd = gpd.GeoDataFrame(segment_day_trips.apply(get_line_string, axis=1)).set_crs(epsg=4326)

AttributeError: ignored

In [ ]:
vmin = segment_day_trips_gpd["day_trips"].min()
vmax = segment_day_trips_gpd["day_trips"].quantile(.95)
vcenter = segment_day_trips_gpd["day_trips"].mean()

print(vmin,vmax,vcenter)
norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)

fig = plt.figure(figsize=(30,21))
ax = plt.axes()
ax.set(facecolor = "black")

def limit_to_bounding_box(gdf, bounding_box):
    return gdf.cx[bounding_box["west"]:bounding_box["east"],bounding_box["south"]:bounding_box["north"]]

# Greater Sydney bounding box
sydney_bb = {
    "north": -33.62,
    "south": -34.086,
    "west": 150.6,
    "east": 151.38,
}

sydney_segment_day_trips_gpd = limit_to_bounding_box(segment_day_trips_gpd, sydney_bb)

sydney_segment_day_trips_gpd.plot(ax=ax, cmap="plasma", column='day_trips', legend=True, norm=norm)

ax.set_title(f"Number of trips per day on Sydney Buses Network on {date:%d %B %Y}", fontsize=30)

plt.savefig("Sydney Buses Network.jpeg")

61.0 61.0 61.0


ValueError: ignored

In [ ]:
# save to file
sydney_segment_day_trips_gpd[["day_trips", "geometry"]].to_csv("data/sydney_bus_network_segments.csv.zip", compression="zip", index=False)

##Create Distance Matrix for all stops using OSRM API


In [ ]:
!pip install polyline
import requests
import folium
import polyline

In [ ]:
url = "http://router.project-osrm.org/route/v1/driving/-117.851364,33.698206;-117.838925,33.672260"
r = requests.get(url)
res = r.json()
res

{'code': 'Ok',
 'routes': [{'distance': 5440,
   'duration': 386.6,
   'geometry': 'gttlEfyhnUpBtCr@gAfT`TxF}DpGkBvO`@fs@``@tAyGyGkGk@aGhQ}k@|EoHhIkHdR|QfJmRxAsLWcS}H{B',
   'legs': [{'distance': 5440,
     'duration': 386.6,
     'steps': [],
     'summary': '',
     'weight': 400}],
   'weight': 400,
   'weight_name': 'routability'}],
 'waypoints': [{'distance': 15.580755,
   'hint': 'tHGyhLZxsoQMAQAASQEAAAAAAAAAAAAAWs1fQprEiEIAAAAAAAAAAIYAAAClAAAAAAAAAAAAAADLTAAAnbv5-EQxAgIcu_n4njECAgAAfwLlTnWV',
   'location': [-117.851235, 33.698116],
   'name': ''},
  {'distance': 31.847501,
   'hint': 'o6aFgNamhYBLAAAAKgAAAAAAAAA_AAAA2iJQQqE95kEAAAAAvmQuQksAAAAqAAAAAAAAAD8AAADLTAAA5-z5-MXLAQKz6_n4RMwBAgAArwHlTnWV',
   'location': [-117.838617, 33.672133],
   'name': 'Carlson Avenue'}]}

In [ ]:
polyline.decode('gttlEfyhnUpBtCr@gAfT`TxF}DpGkBvO`@fs@``@tAyGyGkGk@aGhQ}k@|EoHhIkHdR|QfJmRxAsLWcS}H{B')

[(33.69812, -117.85124),
 (33.69755, -117.85199),
 (33.69729, -117.85163),
 (33.69389, -117.855),
 (33.69264, -117.85405),
 (33.69127, -117.85351),
 (33.68859, -117.85368),
 (33.68023, -117.85897),
 (33.6798, -117.85756),
 (33.68121, -117.85622),
 (33.68143, -117.85493),
 (33.6785, -117.84774),
 (33.67739, -117.84622),
 (33.67574, -117.84472),
 (33.67267, -117.84775),
 (33.67087, -117.84464),
 (33.67042, -117.84246),
 (33.67054, -117.83924),
 (33.67213, -117.83862)]

In [ ]:
url = "http://router.project-osrm.org/route/v1/driving/-117.851364,33.698206;-117.838925,33.672260"
r = requests.get(url)
res = r.json()
res

{'code': 'Ok',
 'routes': [{'distance': 5440,
   'duration': 386.6,
   'geometry': 'gttlEfyhnUpBtCr@gAfT`TxF}DpGkBvO`@fs@``@tAyGyGkGk@aGhQ}k@|EoHhIkHdR|QfJmRxAsLWcS}H{B',
   'legs': [{'distance': 5440,
     'duration': 386.6,
     'steps': [],
     'summary': '',
     'weight': 400}],
   'weight': 400,
   'weight_name': 'routability'}],
 'waypoints': [{'distance': 15.580755,
   'hint': 'tHGyhLZxsoQMAQAASQEAAAAAAAAAAAAAWs1fQprEiEIAAAAAAAAAAIYAAAClAAAAAAAAAAAAAADLTAAAnbv5-EQxAgIcu_n4njECAgAAfwLlTnWV',
   'location': [-117.851235, 33.698116],
   'name': ''},
  {'distance': 31.847501,
   'hint': 'o6aFgNamhYBLAAAAKgAAAAAAAAA_AAAA2iJQQqE95kEAAAAAvmQuQksAAAAqAAAAAAAAAD8AAADLTAAA5-z5-MXLAQKz6_n4RMwBAgAArwHlTnWV',
   'location': [-117.838617, 33.672133],
   'name': 'Carlson Avenue'}]}

In [ ]:
def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = requests.get(url + loc) 
   
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out

In [ ]:
pickup_lon, pickup_lat, dropoff_lon, dropoff_lat = -117.851364,33.698206,-117.838925,33.672260
test_route = get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
test_route

{'distance': 5440,
 'end_point': [33.672133, -117.838617],
 'route': [(33.69812, -117.85124),
  (33.69755, -117.85199),
  (33.69729, -117.85163),
  (33.69389, -117.855),
  (33.69264, -117.85405),
  (33.69127, -117.85351),
  (33.68859, -117.85368),
  (33.68023, -117.85897),
  (33.6798, -117.85756),
  (33.68121, -117.85622),
  (33.68143, -117.85493),
  (33.6785, -117.84774),
  (33.67739, -117.84622),
  (33.67574, -117.84472),
  (33.67267, -117.84775),
  (33.67087, -117.84464),
  (33.67042, -117.84246),
  (33.67054, -117.83924),
  (33.67213, -117.83862)],
 'start_point': [33.698116, -117.851235]}

In [ ]:
test_route['distance']

5440

In [ ]:
def get_map(route):
    
    m = folium.Map(location=[(route['start_point'][0] + route['end_point'][0])/2, 
                             (route['start_point'][1] + route['end_point'][1])/2], 
                   zoom_start=13)

    folium.PolyLine(
        route['route'],
        weight=8,
        color='blue',
        opacity=0.6
    ).add_to(m)

    folium.Marker(
        location=route['start_point'],
        icon=folium.Icon(icon='play', color='green')
    ).add_to(m)

    folium.Marker(
        location=route['end_point'],
        icon=folium.Icon(icon='stop', color='red')
    ).add_to(m)

    return m

In [ ]:
get_map(test_route)

In [ ]:
print(stops_df['geometry'][0])

POINT (-86.5335705337296 39.1642860191081)


In [ ]:
stops_df.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,geometry
0,2759346,45690,Departing Transit Center,"Transit Center bordered by E 3rd, S Walnut and...",39.164286,-86.533571,NaN,0,NaN,NaN,NaN,POINT (-86.53357 39.16429)
1,2927977,45650,Lincoln St and Kirkwood Ave,southeast corner over intersection by stop sign,39.166518,-86.531045,NaN,0,NaN,NaN,2,POINT (-86.53104 39.16652)
2,2927978,45505,Kirkwood Ave and Grant St (Outbound-Eastbound),southwest corner of intersection,39.166486,-86.529943,NaN,0,NaN,NaN,NaN,POINT (-86.52994 39.16649)
3,2927979,45143,Kirkwood Ave and Dunn St,southwest corner of intersection near stop sign,39.166483,-86.528359,NaN,0,NaN,NaN,2,POINT (-86.52836 39.16648)
4,2927980,45597,7th St and Woodlawn Ave at the Indiana Memoria...,located at safety island and protected bicycle...,39.168439,-86.523929,NaN,0,NaN,NaN,1,POINT (-86.52393 39.16844)


In [ ]:
int (len(stops_df))

548

In [ ]:
n_stops = len(stops_df)
n_stops = 5
matrix = [[0]*n_stops]*n_stops
matrix

[[0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [ ]:
matrix = pd.DataFrame(matrix,index= stops_df['stop_name'][:n_stops],columns=stops_df['stop_name'][:n_stops])
matrix

stop_name,Departing Transit Center,Lincoln St and Kirkwood Ave,Kirkwood Ave and Grant St (Outbound-Eastbound),Kirkwood Ave and Dunn St,7th St and Woodlawn Ave at the Indiana Memorial Union
stop_name,,,,,
Departing Transit Center,0,0,0,0,0
Lincoln St and Kirkwood Ave,0,0,0,0,0
Kirkwood Ave and Grant St (Outbound-Eastbound),0,0,0,0,0
Kirkwood Ave and Dunn St,0,0,0,0,0
7th St and Woodlawn Ave at the Indiana Memorial Union,0,0,0,0,0


In [ ]:
for i in range(n_stops):
  for j in range(n_stops):
    if(i != j ):
      result = get_route(stops_df['geometry'][i].x,stops_df['geometry'][i].y,stops_df['geometry'][j].x,stops_df['geometry'][j].y)
      matrix.iloc[i,j] = result['distance']
matrix

stop_name,Departing Transit Center,Lincoln St and Kirkwood Ave,Kirkwood Ave and Grant St (Outbound-Eastbound),Kirkwood Ave and Dunn St,7th St and Woodlawn Ave at the Indiana Memorial Union
stop_name,,,,,
Departing Transit Center,0.0,468.5,570.9,707.4,1313.0
Lincoln St and Kirkwood Ave,717.3,0.0,104.9,241.5,847.1
Kirkwood Ave and Grant St (Outbound-Eastbound),810.9,337.6,0.0,136.5,742.1
Kirkwood Ave and Dunn St,947.4,449.5,136.5,0.0,605.6
7th St and Woodlawn Ave at the Indiana Memorial Union,1550.0,1053.0,739.1,602.6,0.0


In [ ]:
matrix

stop_name,Departing Transit Center,Lincoln St and Kirkwood Ave,Kirkwood Ave and Grant St (Outbound-Eastbound),Kirkwood Ave and Dunn St,7th St and Woodlawn Ave at the Indiana Memorial Union
stop_name,,,,,
Departing Transit Center,0.0,468.5,570.9,707.4,1313.0
Lincoln St and Kirkwood Ave,717.3,0.0,104.9,241.5,847.1
Kirkwood Ave and Grant St (Outbound-Eastbound),810.9,337.6,0.0,136.5,742.1
Kirkwood Ave and Dunn St,947.4,449.5,136.5,0.0,605.6
7th St and Woodlawn Ave at the Indiana Memorial Union,1550.0,1053.0,739.1,602.6,0.0


In [ ]:
matrix.to_csv("/content/drive/My Drive/"+"Distance Matrix : "+gtfs_data+".csv")

In [ ]:
stops_df.iloc[:n_stops]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,geometry
0,2759346,45690,Departing Transit Center,"Transit Center bordered by E 3rd, S Walnut and...",39.164286,-86.533571,NaN,0,NaN,NaN,NaN,POINT (-86.53357 39.16429)
1,2927977,45650,Lincoln St and Kirkwood Ave,southeast corner over intersection by stop sign,39.166518,-86.531045,NaN,0,NaN,NaN,2,POINT (-86.53104 39.16652)
2,2927978,45505,Kirkwood Ave and Grant St (Outbound-Eastbound),southwest corner of intersection,39.166486,-86.529943,NaN,0,NaN,NaN,NaN,POINT (-86.52994 39.16649)
3,2927979,45143,Kirkwood Ave and Dunn St,southwest corner of intersection near stop sign,39.166483,-86.528359,NaN,0,NaN,NaN,2,POINT (-86.52836 39.16648)
4,2927980,45597,7th St and Woodlawn Ave at the Indiana Memoria...,located at safety island and protected bicycle...,39.168439,-86.523929,NaN,0,NaN,NaN,1,POINT (-86.52393 39.16844)


In [ ]:
stops_df.iloc[:n_stops].to_csv("/content/drive/My Drive/"+"Stops data : "+gtfs_data+".csv")